# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d4cf475169-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("Olympic explorative search") 

Consider the following exploratory information need:

> investigate the results, in terms of participations and competitions won, in the past editions of the olympics between France and Germany

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q159821`  | Olympic games | node      |
| `wd:Q10843402`| Swimmer       | node |
| `wd:Q2066131` | Athlete       | node |
| `wd:Q142`     | France        | node |
| `wd:Q183`     | Germany       | node |


Also consider


```
?a wdt:P106/wdt:P279 wd:Q2066131
```

is the BGP to retrieve all instances of **athlete**


```
?p wdt:P27 wd:Q142
```

is the BGP to retrieve all **citizens of France**

## Workload Goals

1. Identify the BGP for participating or winning some competition

2. Identify the BGP for different sports or types of competitions

3. Is there some French or German Athlete participating to any olympic event with double citizenship?

4. Compare number of participations and winning events across different dimensions
 
   4.1 In which specific sport France has more winners than Germany
   
   4.2 Who has the highest number of female winners
   
   4.3 For each country , which are the atheletes with the highest number of participations


In [1]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '929008')]


1

#### Comment
From how the texts of the tasks is written I suppose, when needed, to retrieve the results only for France and Germany (e.g. in 4.2 and 4.3) 

## Task 1
1. Identify the BGP for participating or winning some competition

#### Comment
I suppose olympics competitions

In [12]:
#Let's check the incoming properties for "Olympic Games"
queryString = """
SELECT DISTINCT ?prop ?p
WHERE { 

?sub ?p wd:Q159821 .

?p <http://schema.org/name> ?prop .
#?sub <http://schema.org/name> ?sname .

} 
"""

print("Results")
run_query(queryString)

Results
[('prop', 'participant in'), ('p', 'http://www.wikidata.org/prop/direct/P1344')]
[('prop', 'named after'), ('p', 'http://www.wikidata.org/prop/direct/P138')]
[('prop', 'based on'), ('p', 'http://www.wikidata.org/prop/direct/P144')]
[('prop', 'part of the series'), ('p', 'http://www.wikidata.org/prop/direct/P179')]
[('prop', 'has parts of the class'), ('p', 'http://www.wikidata.org/prop/direct/P2670')]
[('prop', "category's main topic"), ('p', 'http://www.wikidata.org/prop/direct/P301')]
[('prop', 'instance of'), ('p', 'http://www.wikidata.org/prop/direct/P31')]
[('prop', 'part of'), ('p', 'http://www.wikidata.org/prop/direct/P361')]
[('prop', 'opposite of'), ('p', 'http://www.wikidata.org/prop/direct/P461')]
[('prop', 'main subject'), ('p', 'http://www.wikidata.org/prop/direct/P921')]
[('prop', 'category combines topics'), ('p', 'http://www.wikidata.org/prop/direct/P971')]


11

In [10]:
#Let's check the incoming properties for "Olympic Games" looking also at the staring node name
queryString = """
SELECT DISTINCT ?sname ?prop ?p ?sub
WHERE { 

?sub ?p wd:Q159821 .

?p <http://schema.org/name> ?prop .
?sub <http://schema.org/name> ?sname .

} 
"""

print("Results")
run_query(queryString)

Results
[('sname', 'fencing at the Summer Olympics'), ('prop', 'part of'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('sub', 'http://www.wikidata.org/entity/Q29762')]
[('sname', 'weightlifting at the Summer Olympics'), ('prop', 'part of'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('sub', 'http://www.wikidata.org/entity/Q733352')]
[('sname', 'Category:Ecuador at the Summer Olympics'), ('prop', 'category combines topics'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('sub', 'http://www.wikidata.org/entity/Q9621038')]
[('sname', 'sport climbing at the Summer Olympics'), ('prop', 'part of'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('sub', 'http://www.wikidata.org/entity/Q26832970')]
[('sname', 'gymnastics at the Summer Olympics'), ('prop', 'part of'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('sub', 'http://www.wikidata.org/entity/Q663317')]
[('sname', 'cycling at the Summer Olympics'), ('prop', 'part of'), ('p', 'http://www.wikidata.org/prop/

151

In [13]:
#Let's check the outgoing properties for "Olympic Games"
queryString = """
SELECT DISTINCT ?prop ?p
WHERE { 

wd:Q159821 ?p ?obj .

?p <http://schema.org/name> ?prop .

} 
"""

print("Results")
run_query(queryString)

Results
[('prop', 'described by source'), ('p', 'http://www.wikidata.org/prop/direct/P1343')]
[('prop', 'image'), ('p', 'http://www.wikidata.org/prop/direct/P18')]
[('prop', 'VIAF ID'), ('p', 'http://www.wikidata.org/prop/direct/P214')]
[('prop', 'event interval'), ('p', 'http://www.wikidata.org/prop/direct/P2257')]
[('prop', 'GND ID'), ('p', 'http://www.wikidata.org/prop/direct/P227')]
[('prop', 'YSO ID'), ('p', 'http://www.wikidata.org/prop/direct/P2347')]
[('prop', 'Library of Congress authority ID'), ('p', 'http://www.wikidata.org/prop/direct/P244')]
[('prop', 'BabelNet ID'), ('p', 'http://www.wikidata.org/prop/direct/P2581')]
[('prop', 'IdRef ID'), ('p', 'http://www.wikidata.org/prop/direct/P269')]
[('prop', 'subclass of'), ('p', 'http://www.wikidata.org/prop/direct/P279')]
[('prop', 'instance of'), ('p', 'http://www.wikidata.org/prop/direct/P31')]
[('prop', 'Quora topic ID'), ('p', 'http://www.wikidata.org/prop/direct/P3417')]
[('prop', 'Commons category'), ('p', 'http://www.wiki

29

#### Comment 
From the results I suppose that the property "part of" (wdt:P361) is the property to identify the various disciplines

In [7]:
#Let's check the properties for an athlete
queryString = """
SELECT DISTINCT ?prop ?p
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .

    ?p <http://schema.org/name> ?prop .

    FILTER((REGEX(?prop, "(?i)part.*")) || (REGEX(?prop, "(?i)win.*"))) .  

} 
"""

print("Results")
run_query(queryString)

Results
[('prop', 'participant in'), ('p', 'http://www.wikidata.org/prop/direct/P1344')]
[('prop', 'unmarried partner'), ('p', 'http://www.wikidata.org/prop/direct/P451')]
[('prop', 'participant'), ('p', 'http://www.wikidata.org/prop/direct/P710')]
[('prop', 'participating team'), ('p', 'http://www.wikidata.org/prop/direct/P1923')]
[('prop', 'member of political party'), ('p', 'http://www.wikidata.org/prop/direct/P102')]
[('prop', 'partner in business or sport'), ('p', 'http://www.wikidata.org/prop/direct/P1327')]
[('prop', 'World Rowing ID (numeric format)'), ('p', 'http://www.wikidata.org/prop/direct/P2091')]
[('prop', 'World Rowing ID (UUID format)'), ('p', 'http://www.wikidata.org/prop/direct/P8990')]
[('prop', 'part of'), ('p', 'http://www.wikidata.org/prop/direct/P361')]
[('prop', 'winner'), ('p', 'http://www.wikidata.org/prop/direct/P1346')]
[('prop', 'Wikipedia Library partner ID'), ('p', 'http://www.wikidata.org/prop/direct/P8584')]
[('prop', 'International Mathematical Olympi

24

#### Comment
We have discoverd the properties:
    
| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1344`   | partecipant in   | predicate |
| `wdt:P710`     | partecipant   | predicate |
| `wdt:P1346`    | winner    | predicate |
| `wdt:P361`    | part of    | predicate |
| `wdt:P31`    | instance of    | predicate |

That can be usefull.
Let's see the nodes connected to them.

In [15]:
#Let's check if there are properties for athlete 
queryString = """
SELECT DISTINCT ?prop ?oname ?obj
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .
?obj wdt:P361 wd:Q159821.

    FILTER(?p = wdt:P1344 || ?p = wdt:P710 || ?p = wdt:P1346) .  

?p <http://schema.org/name> ?prop .
?obj <http://schema.org/name> ?oname .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
Empty


0

#### Comment
There are no athlete related nodes directly connected to "Olympic Games" nodes through "part of" property, so I need to find another path that connects the disciple with a partecipant/winner

In [16]:
#Let's retrieve a sample of nodes (frist 20 results) connected to the property "partecipant " starting from an athlete
queryString = """
SELECT DISTINCT ?prop ?oname ?obj
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .

    FILTER(?p = wdt:P710) .  

?p <http://schema.org/name> ?prop .
?obj <http://schema.org/name> ?oname .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('prop', 'participant'), ('oname', "2019 FIFA Women's World Cup"), ('obj', 'http://www.wikidata.org/entity/Q4630361')]
[('prop', 'participant'), ('oname', "2007 FIFA Women's World Cup"), ('obj', 'http://www.wikidata.org/entity/Q367805')]
[('prop', 'participant'), ('oname', '2012 Summer Olympics'), ('obj', 'http://www.wikidata.org/entity/Q8577')]
[('prop', 'participant'), ('oname', '2016 Summer Olympics'), ('obj', 'http://www.wikidata.org/entity/Q8613')]
[('prop', 'participant'), ('oname', '2008 Summer Olympics'), ('obj', 'http://www.wikidata.org/entity/Q8567')]
[('prop', 'participant'), ('oname', "FIFA Women's World Cup"), ('obj', 'http://www.wikidata.org/entity/Q19323')]
[('prop', 'participant'), ('oname', 'Eastern Mediterranean Championship Tennis'), ('obj', 'http://www.wikidata.org/entity/Q28739880')]
[('prop', 'participant'), ('oname', "Panhellenic men's and women's tennis championship"), ('obj', 'http://www.wikidata.org/entity/Q27779355')]
[('prop', 'participant'), ('onam

20

In [17]:
#Let's retrieve a sample of nodes connected to the property "partecipant in" starting from an athlete

queryString = """
SELECT DISTINCT ?prop ?oname ?obj
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .

    FILTER(?p = wdt:P1344) .  

?p <http://schema.org/name> ?prop .
?obj <http://schema.org/name> ?oname .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('prop', 'participant in'), ('oname', "shooting at the 1992 Summer Olympics – men's 50 metre rifle three positions"), ('obj', 'http://www.wikidata.org/entity/Q17088224')]
[('prop', 'participant in'), ('oname', "2014 Asian Junior Badminton Championships – men's singles"), ('obj', 'http://www.wikidata.org/entity/Q15741854')]
[('prop', 'participant in'), ('oname', "shooting at the 1996 Summer Olympics – women's 50 metre rifle three positions"), ('obj', 'http://www.wikidata.org/entity/Q7500802')]
[('prop', 'participant in'), ('oname', '2013 Asian Wrestling Championships'), ('obj', 'http://www.wikidata.org/entity/Q15720586')]
[('prop', 'participant in'), ('oname', "cross-country skiing at the 2014 Winter Paralympics – women's 15 km free"), ('obj', 'http://www.wikidata.org/entity/Q15920784')]
[('prop', 'participant in'), ('oname', 'Cross-country skiing at the 2014 Winter Paralympics - Sprint 1 km (female)'), ('obj', 'http://www.wikidata.org/entity/Q15920827')]
[('prop', 'participant

20

In [18]:
#Let's retrieve a sample of nodes connected to the property "winner" starting from an athlete
queryString = """
SELECT DISTINCT ?prop ?oname ?obj
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .

    FILTER(?p = wdt:P1346) .  

?p <http://schema.org/name> ?prop .
?obj <http://schema.org/name> ?oname .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('prop', 'winner'), ('oname', "2016 World Wrestling Championships – women's freestyle 55 kg"), ('obj', 'http://www.wikidata.org/entity/Q28406503')]
[('prop', 'winner'), ('oname', "2018 World Wrestling Championships – Women's freestyle 55 kg"), ('obj', 'http://www.wikidata.org/entity/Q57609226')]
[('prop', 'winner'), ('oname', "FIFA Women's World Cup"), ('obj', 'http://www.wikidata.org/entity/Q19323')]
[('prop', 'winner'), ('oname', '2007 Southeast Asian Games'), ('obj', 'http://www.wikidata.org/entity/Q1209340')]
[('prop', 'winner'), ('oname', '2016 Supercopa de España de Baloncesto'), ('obj', 'http://www.wikidata.org/entity/Q24908675')]
[('prop', 'winner'), ('oname', 'North American Boxing Federation'), ('obj', 'http://www.wikidata.org/entity/Q1628588')]
[('prop', 'winner'), ('oname', 'WWE Raw Tag Team Championship'), ('obj', 'http://www.wikidata.org/entity/Q1059464')]
[('prop', 'winner'), ('oname', 'Amsterdam Marathon'), ('obj', 'http://www.wikidata.org/entity/Q478328')]
[('

15

In [20]:
#Let's check the name of "Olympic Games" (wd:Q159821)
queryString = """
SELECT ?oname
WHERE { 

wd:Q159821 <http://schema.org/name> ?oname .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('oname', 'Summer Olympic Games')]


1

#### Comment

So we are talking about Summer Olympic Games when we use (wd:Q159821).

Let's try to print the properies related to "shooting at the 1996 Summer Olympics – women's 50 metre rifle three positions" (wd:Q7500802) that is probably a competition related to the Olympic games

In [22]:
#Let's try to print the properies related to "shooting at the 1996 Summer Olympics – women's 50 metre rifle three positions" (wd:Q7500802)
queryString = """
SELECT DISTINCT ?prop ?oname ?p ?obj
WHERE { 
  
wd:Q7500802 ?p ?obj .

?p <http://schema.org/name> ?prop .
OPTIONAL {?obj <http://schema.org/name> ?oname } .
}
"""

print("Results")
run_query(queryString)

Results
[('prop', 'instance of'), ('oname', 'Olympic sporting event'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q18536594')]
[('prop', 'sport'), ('oname', 'shooting sport'), ('p', 'http://www.wikidata.org/prop/direct/P641'), ('obj', 'http://www.wikidata.org/entity/Q206989')]
[('prop', 'followed by'), ('oname', "shooting at the 2000 Summer Olympics – women's 50 metre rifle three positions"), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('obj', 'http://www.wikidata.org/entity/Q7500819')]
[('prop', 'part of'), ('oname', 'shooting at the 1996 Summer Olympics'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('obj', 'http://www.wikidata.org/entity/Q774254')]
[('prop', 'location'), ('oname', 'Wolf Creek Shooting Complex'), ('p', 'http://www.wikidata.org/prop/direct/P276'), ('obj', 'http://www.wikidata.org/entity/Q8029829')]
[('prop', 'winner'), ('oname', 'Aleksandra Ivošev'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('obj'

11

#### Comment
We have discovered many interesting things:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P641`   | sport   | predicate |
| `wdt:P361`     | part of   | predicate |
| `wdt:P1346`    | winner    | predicate |
| `wdt:P585`    | point in time    | predicate |
| `wd:Q18536594`    | Olympic sporting event    | node |

And so that the "sport" property connects a competition with its belonging type of sport.  
With the property "instance of" we can read the event (and so for example Olympic sporting event) in which the competiton is organized.  
With the "part of" property is specified the father event and so the event without the speciality (e.g. women's 50 metre rifle three positions).  
"winner" specifies the athlete that won that speciality. 

In [25]:
# Let's try to find 10 athletes that are "winner" of speciality 
queryString = """
SELECT DISTINCT ?aname ?prop ?oname ?a ?p ?obj
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .
?obj wdt:P31 wd:Q18536594 .

    FILTER(?p = wdt:P1346) .  

?p <http://schema.org/name> ?prop .
?a <http://schema.org/name> ?aname .
?obj <http://schema.org/name> ?oname .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
Empty


0

#### Comment
Probably the "winner" property doesn't connect an athlete with the Olympic competitions that he/she won. 

In [26]:
# Let's try to find 10 athletes that are "partecipant in" a speciality 
queryString = """
SELECT DISTINCT ?aname ?prop ?oname ?a ?p ?obj
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .
?obj wdt:P31 wd:Q18536594 .

    FILTER(?p = wdt:P1344) .  

?p <http://schema.org/name> ?prop .
?a <http://schema.org/name> ?aname .
?obj <http://schema.org/name> ?oname .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('aname', 'Tamás Wichmann'), ('prop', 'participant in'), ('oname', "canoeing at the 1968 Summer Olympics – men's C-2 1000 metres"), ('a', 'http://www.wikidata.org/entity/Q1464708'), ('p', 'http://www.wikidata.org/prop/direct/P1344'), ('obj', 'http://www.wikidata.org/entity/Q17078738')]
[('aname', 'Mikhail Zamotin'), ('prop', 'participant in'), ('oname', "canoeing at the 1968 Summer Olympics – men's C-2 1000 metres"), ('a', 'http://www.wikidata.org/entity/Q4185947'), ('p', 'http://www.wikidata.org/prop/direct/P1344'), ('obj', 'http://www.wikidata.org/entity/Q17078738')]
[('aname', 'Serghei Covaliov'), ('prop', 'participant in'), ('oname', "canoeing at the 1968 Summer Olympics – men's C-2 1000 metres"), ('a', 'http://www.wikidata.org/entity/Q2614951'), ('p', 'http://www.wikidata.org/prop/direct/P1344'), ('obj', 'http://www.wikidata.org/entity/Q17078738')]
[('aname', 'Gyula Petrikovics'), ('prop', 'participant in'), ('oname', "canoeing at the 1968 Summer Olympics – men's C-2 1000

10

In [29]:
# Let's try to find 10 athletes that are "partecipant" a speciality 
queryString = """
SELECT DISTINCT ?aname ?prop ?oname ?a ?p ?obj
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .
?obj wdt:P31 wd:Q18536594 .

    FILTER(?p = wdt:P710) .  

?p <http://schema.org/name> ?prop .
?a <http://schema.org/name> ?aname .
?obj <http://schema.org/name> ?oname .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('aname', 'Saud Al-Zaabi\u200e'), ('prop', 'participant'), ('oname', "athletics at the 2016 Summer Olympics – men's 1500 metres"), ('a', 'http://www.wikidata.org/entity/Q26789004'), ('p', 'http://www.wikidata.org/prop/direct/P710'), ('obj', 'http://www.wikidata.org/entity/Q26219859')]
[('aname', 'Shanice van de Sanden'), ('prop', 'participant'), ('oname', "football at the 2020 Summer Olympics – women's tournament"), ('a', 'http://www.wikidata.org/entity/Q2277010'), ('p', 'http://www.wikidata.org/prop/direct/P710'), ('obj', 'http://www.wikidata.org/entity/Q62589205')]
[('aname', 'Jackie Groenen'), ('prop', 'participant'), ('oname', "football at the 2020 Summer Olympics – women's tournament"), ('a', 'http://www.wikidata.org/entity/Q460559'), ('p', 'http://www.wikidata.org/prop/direct/P710'), ('obj', 'http://www.wikidata.org/entity/Q62589205')]
[('aname', 'Peter Bol'), ('prop', 'participant'), ('oname', "athletics at the 2016 Summer Olympics – men's 800 metres"), ('a', 'http://ww

4

#### Comment
So it seems that te principal property for the partecipation to an Olympic sport event is "partecipant in" (wdt:P1344), in fact "partecipant" (wdt:710) only retrieves 4 results.  
To obtain the information about the BGP we need to do something different, so let's have a try.

In [34]:
# Let's try to find 10 athletes that won an olympic sport event, so starting from the event node let's see the node connected to it through "winner" property 
queryString = """
SELECT DISTINCT ?aname ?winner ?oname ?a ?win ?obj
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .
?obj wdt:P31 wd:Q18536594 ;
    ?win ?a .

FILTER(?p = wdt:P710 || ?p = wdt:P1344) . 
FILTER(?win = wdt:P1346) .

#?p <http://schema.org/name> ?prop .
?win <http://schema.org/name> ?winner .
?a <http://schema.org/name> ?aname .
?obj <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('aname', 'Benjamin Savšek'), ('winner', 'winner'), ('oname', "canoeing at the 2020 Summer Olympics – men's slalom C-1"), ('a', 'http://www.wikidata.org/entity/Q4889209'), ('win', 'http://www.wikidata.org/prop/direct/P1346'), ('obj', 'http://www.wikidata.org/entity/Q76711065')]
[('aname', 'Kiran Badloe'), ('winner', 'winner'), ('oname', "sailing at the 2020 Summer Olympics – men's RS:X"), ('a', 'http://www.wikidata.org/entity/Q68701450'), ('win', 'http://www.wikidata.org/prop/direct/P1346'), ('obj', 'http://www.wikidata.org/entity/Q65229885')]
[('aname', 'Ariarne Titmus'), ('winner', 'winner'), ('oname', "swimming at the 2020 Summer Olympics – women's 200 metre freestyle"), ('a', 'http://www.wikidata.org/entity/Q33320896'), ('win', 'http://www.wikidata.org/prop/direct/P1346'), ('obj', 'http://www.wikidata.org/entity/Q64514018')]
[('aname', 'Allyson Felix'), ('winner', 'winner'), ('oname', "athletics at the 2012 Summer Olympics – women's 200 metres"), ('a', 'http://www.wikidata

25

#### Comment

We have obtained the desired result and so we can write the BGP that satisfies the task.

In [40]:
# Let's try to find 10 athletes that partecipated in or won or both an olympic sport event 
queryString = """
SELECT DISTINCT ?aname (GROUP_CONCAT(DISTINCT ?prop ; separator = ", ") AS ?do) ?oname
WHERE { 
{

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .
?obj wdt:P31 wd:Q18536594 .

FILTER(?p = wdt:P710 || ?p = wdt:P1344) .

}
UNION
{

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?rel ?obj .
?obj wdt:P31 wd:Q18536594 ;
    ?p ?a .
FILTER(?p = wdt:P1346) .

}
?p <http://schema.org/name> ?prop .
#?win <http://schema.org/name> ?winner .
?a <http://schema.org/name> ?aname .
?obj <http://schema.org/name> ?oname .
}
GROUP BY ?aname ?oname
ORDER BY ASC (?aname)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('aname', "A'ja Wilson"), ('do', 'participant in'), ('oname', "basketball at the 2020 Summer Olympics – women's tournament")]
[('aname', 'A. Kordonis'), ('do', 'participant in'), ('oname', 'rowing at the 1906 Intercalated Games – 6 men naval boat')]
[('aname', 'A. Lambrecht'), ('do', 'participant in'), ('oname', "rowing at the 1928 Summer Olympics – men's eight")]
[('aname', 'Aage Larsen'), ('do', 'participant in'), ('oname', "rowing at the 1948 Summer Olympics – men's double sculls")]
[('aname', 'Aage Larsen'), ('do', 'participant in'), ('oname', "rowing at the 1952 Summer Olympics – men's double sculls")]
[('aname', 'Aaliyah Edwards'), ('do', 'participant in'), ('oname', "basketball at the 2020 Summer Olympics – women's tournament")]
[('aname', 'Aarne Lindroos'), ('do', 'participant in'), ('oname', "rowing at the 1988 Summer Olympics – men's coxless pair")]
[('aname', 'Aarne Lindroos'), ('do', 'participant in'), ('oname', "rowing at the 1984 Summer Olympics – men's double sc

25

#### Comment
As we can see for each athlete it's printed if it has "partecipated in" and eventually won an Olympic sport event printing also the relative competition

The corresponding BGP is:
```

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .
?obj wdt:P31 wd:Q18536594 .

FILTER(?p = wdt:P710 || ?p = wdt:P1344) .

}
UNION
{

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?rel ?obj .
?obj wdt:P31 wd:Q18536594 ;
    ?p ?a .
FILTER(?p = wdt:P1346) .

```

Or

```
?a wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P710|wdt:P1344 ?obj .
?obj wdt:P31 wd:Q18536594 .

}
UNION
{

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?rel ?obj .
?obj wdt:P31 wd:Q18536594 ;
    wdt:P1346 ?a .

```

The filter and the GROUP_CONCAT constructs are used only for having a better visualization of the results. 

## Task 2
2. Identify the BGP for different sports or types of competitions

#### Comment
I don't know if I understand well the text of the task but I'm going to print the different Olympic sports or different types of competition

In [42]:
# For each competition let's print the sport and the type of competition let's limiti the results at the first 25, in fact we need only an example
queryString = """
SELECT DISTINCT ?prop ?result
WHERE { 

?obj wdt:P31 wd:Q18536594 ;
    ?p ?res .
    
    FILTER(?p = wdt:P641 || ?p = wdt:P361) .

?p <http://schema.org/name> ?prop .
?res <http://schema.org/name> ?result .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('prop', 'part of'), ('result', 'diving at the 1932 Summer Olympics')]
[('prop', 'part of'), ('result', 'diving at the 1920 Summer Olympics')]
[('prop', 'part of'), ('result', 'field hockey at the 2008 Summer Olympics')]
[('prop', 'part of'), ('result', 'diving at the 1984 Summer Olympics')]
[('prop', 'part of'), ('result', 'water motorsports at the 1908 Summer Olympics')]
[('prop', 'part of'), ('result', 'equestrian at the 1996 Summer Olympics')]
[('prop', 'part of'), ('result', 'wrestling at the 1936 Summer Olympics')]
[('prop', 'part of'), ('result', 'gymnastics at the 1936 Summer Olympics')]
[('prop', 'part of'), ('result', 'wrestling at the 1948 Summer Olympics')]
[('prop', 'part of'), ('result', 'ice hockey at the 1984 Winter Olympics')]
[('prop', 'part of'), ('result', 'rowing at the 1988 Summer Olympics')]
[('prop', 'part of'), ('result', 'rowing at the 1960 Summer Olympics')]
[('prop', 'part of'), ('result', 'rowing at the 1976 Summer Olympics')]
[('prop', 'part of'),

25

#### Comment
So the BGP to retrieve the different sports or type of competitions is:
```

?obj wdt:P31 wd:Q18536594 ;
    ?p ?res .
    
    FILTER(?p = wdt:P641 || ?p = wdt:P361) .

```

Or

```

?obj wdt:P31 wd:Q18536594 ;
    wdt:P641|wdt:P361 ?res .

```
The filter construct is used only for having a better visualization of the results, in fact it can print the name of the property that connects the two nodes. 

## Task 3
3. Is there some French or German Athlete participating to any olympic event with double citizenship?

In [51]:
# Let's try to find if any athlete has more than one citizenship counting the nodes reached from a "nationality" (wdt:P27) property
queryString = """

SELECT DISTINCT ?aname ?num WHERE {

?at wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj ;
    wdt:P27 ?n .
?obj wdt:P31 wd:Q18536594 .

FILTER(?at = ?a && ?num > 1)
{
    SELECT ?a (COUNT(DISTINCT ?nat) AS ?num) WHERE { 

        ?a wdt:P106/wdt:P279 wd:Q2066131 ;
            ?pr ?obj ;
            wdt:P27 ?nat .
        ?obj wdt:P31 wd:Q18536594 .

        FILTER(?pr = wdt:P710 || ?pr = wdt:P1344) .
    }
    GROUP BY ?a
}

FILTER(?p = wdt:P710 || ?p = wdt:P1344) .
FILTER(?n = wd:Q142 || ?n = wd:Q183) .

?p <http://schema.org/name> ?prop .
?at <http://schema.org/name> ?aname .
?obj <http://schema.org/name> ?oname .
}
ORDER BY ASC (?aname)
"""

print("Results")
run_query(queryString)

Results
[('aname', 'Achim Hill'), ('num', '2')]
[('aname', 'Adrian Hoesch'), ('num', '2')]
[('aname', 'Albrecht Müller'), ('num', '2')]
[('aname', 'Albrecht Wehselau'), ('num', '2')]
[('aname', 'Aleksander Cichoń'), ('num', '2')]
[('aname', 'Alexander Schuck'), ('num', '2')]
[('aname', 'Ampelio Sartor'), ('num', '2')]
[('aname', 'Andrea Eife'), ('num', '2')]
[('aname', 'Andrea Heim'), ('num', '2')]
[('aname', 'Andrea Pollack'), ('num', '2')]
[('aname', 'Andreas Ihle'), ('num', '2')]
[('aname', 'Andreas Knebel'), ('num', '2')]
[('aname', 'Andreas Schröder'), ('num', '2')]
[('aname', 'Andreas Stähle'), ('num', '2')]
[('aname', 'Andreas Wank'), ('num', '2')]
[('aname', 'André Wohllebe'), ('num', '2')]
[('aname', 'André-Pierre Gignac'), ('num', '2')]
[('aname', 'Anett Schuck'), ('num', '2')]
[('aname', 'Angela Franke'), ('num', '2')]
[('aname', 'Angelika Bahmann'), ('num', '2')]
[('aname', 'Anita Nüßner'), ('num', '2')]
[('aname', 'Anke Behmer'), ('num', '2')]
[('aname', 'Anke Möhring'), (

566

#### Comment
This is the list of French and Germans athlete that have a double citizenship.  
So to fulfil the task: yes there are many French and German athletes with double citizeship.

## Task 4
4. Compare number of participations and winning events across different dimensions

### Task 4.1
4.1 In which specific sport France has more winners than Germany

#### Comment
I suppose that "more winners" is different from "more win" and so I'm going to retrieve the sports in which France has more winners (athletes) than German

In [17]:
# Let's print the the number of distinct French winners for each sport 
queryString = """
SELECT ?sport (COUNT(DISTINCT ?at) AS ?numf) WHERE {

?obj wdt:P31 wd:Q18536594 ;
    ?sp ?s ;
    ?rel ?at .    
?at wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?nf .
    
FILTER(?nf = wd:Q142) .
FILTER(?sp = wdt:P641 && ?rel = wdt:P1346) .

?s <http://schema.org/name> ?sport .
?at <http://schema.org/name> ?aname .
}
ORDER BY ASC (?sport)
"""
print("Results")
run_query(queryString)

Results
[('sport', 'BMX'), ('numf', '2')]
[('sport', 'Greco-Roman wrestling'), ('numf', '2')]
[('sport', 'archery'), ('numf', '6')]
[('sport', 'athletics'), ('numf', '10')]
[('sport', 'biathlon'), ('numf', '1')]
[('sport', 'boxing'), ('numf', '4')]
[('sport', 'canoe slalom'), ('numf', '1')]
[('sport', 'canoeing and kayaking'), ('numf', '3')]
[('sport', 'croquet'), ('numf', '2')]
[('sport', 'cycle sport'), ('numf', '10')]
[('sport', 'equestrian sport'), ('numf', '1')]
[('sport', 'fencing'), ('numf', '35')]
[('sport', 'judo'), ('numf', '13')]
[('sport', 'mountain biking'), ('numf', '2')]
[('sport', 'road bicycle racing'), ('numf', '24')]
[('sport', 'rowing'), ('numf', '7')]
[('sport', 'sailing'), ('numf', '8')]
[('sport', 'shooting sport'), ('numf', '12')]
[('sport', 'swimming'), ('numf', '6')]
[('sport', 'tennis'), ('numf', '6')]
[('sport', 'track cycling'), ('numf', '52')]
[('sport', 'wrestling'), ('numf', '2')]


22

In [15]:
# Let's print the the number of distinct German winners for each sport 
queryString = """
SELECT ?sport (COUNT(DISTINCT ?at) AS ?numg) WHERE {

?obj wdt:P31 wd:Q18536594 ;
    ?sp ?s ;
    ?rel ?at .    
?at wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?ng .
    
FILTER(?ng = wd:Q183) .
FILTER(?sp = wdt:P641 && ?rel = wdt:P1346) .

?s <http://schema.org/name> ?sport .
?at <http://schema.org/name> ?aname .
}
ORDER BY ASC (?sport)
"""
print("Results")
run_query(queryString)

Results
[('sport', 'Greco-Roman wrestling'), ('numg', '7')]
[('sport', 'Nordic combined'), ('numg', '1')]
[('sport', 'Nordic skiing'), ('numg', '3')]
[('sport', 'artistic gymnastics'), ('numg', '1')]
[('sport', 'athletics'), ('numg', '57')]
[('sport', 'beach volleyball'), ('numg', '2')]
[('sport', 'biathlon'), ('numg', '1')]
[('sport', 'bobsleigh'), ('numg', '1')]
[('sport', 'boxing'), ('numg', '9')]
[('sport', 'canoe sprint'), ('numg', '2')]
[('sport', 'canoeing and kayaking'), ('numg', '7')]
[('sport', 'cycle sport'), ('numg', '11')]
[('sport', 'diving'), ('numg', '5')]
[('sport', 'equestrian sport'), ('numg', '1')]
[('sport', 'eventing'), ('numg', '1')]
[('sport', 'fencing'), ('numg', '12')]
[('sport', 'gymnastics'), ('numg', '1')]
[('sport', 'judo'), ('numg', '5')]
[('sport', 'luge'), ('numg', '2')]
[('sport', 'modern pentathlon'), ('numg', '1')]
[('sport', 'mountain biking'), ('numg', '1')]
[('sport', 'road bicycle racing'), ('numg', '30')]
[('sport', 'rowing'), ('numg', '40')]
[(

31

In [25]:
# Let's try to print in which sports France has more winners than Germany
queryString = """
SELECT ?sport (COUNT(DISTINCT ?atf) AS ?frenchwins) (COUNT(DISTINCT ?atg) AS ?germanwins) WHERE {

{
?obj wdt:P31 wd:Q18536594 ;
    ?sp ?s ;
    ?rel ?atf .    
?atf wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?nf .
    
FILTER(?nf = wd:Q142) .
FILTER(?sp = wdt:P641 && ?rel = wdt:P1346) .

}UNION{

?obj wdt:P31 wd:Q18536594 ;
    ?sp ?s ;
    ?rel ?atg .    
?atg wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?ng .
    
FILTER(?ng = wd:Q183) .
FILTER(?sp = wdt:P641 && ?rel = wdt:P1346) .

}

?s <http://schema.org/name> ?sport .
#?at <http://schema.org/name> ?aname .
#?obj <http://schema.org/name> ?oname .
}
GROUP BY ?sport
HAVING(COUNT(DISTINCT ?atf) > COUNT(DISTINCT ?atg))
ORDER BY ASC (?sport)
"""

print("Results")
run_query(queryString)

Results
[('sport', 'BMX'), ('frenchwins', '2'), ('germanwins', '0')]
[('sport', 'archery'), ('frenchwins', '6'), ('germanwins', '0')]
[('sport', 'canoe slalom'), ('frenchwins', '1'), ('germanwins', '0')]
[('sport', 'croquet'), ('frenchwins', '2'), ('germanwins', '0')]
[('sport', 'fencing'), ('frenchwins', '35'), ('germanwins', '12')]
[('sport', 'judo'), ('frenchwins', '13'), ('germanwins', '5')]
[('sport', 'mountain biking'), ('frenchwins', '2'), ('germanwins', '1')]
[('sport', 'sailing'), ('frenchwins', '8'), ('germanwins', '3')]
[('sport', 'tennis'), ('frenchwins', '6'), ('germanwins', '4')]


9

#### Comment
The last query shows each sport in which there are more French winners than German ones printing the respective numbers

### Task 4.2
4.2 Who has the highest number of female winners

#### Comment
Remember that I supposed to compare the number of female winners which citizenship is French or German

In [28]:
#Let's search for the gender property of an athlete
queryString = """
SELECT DISTINCT ?prop ?gender ?p ?obj
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    ?p ?obj .

    ?p <http://schema.org/name> ?prop .
    ?obj <http://schema.org/name> ?gender

    FILTER((REGEX(?prop, "(?i)gender.*"))) .  

} 
"""

print("Results")
run_query(queryString)

Results
[('prop', 'sex or gender'), ('gender', 'female'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('obj', 'http://www.wikidata.org/entity/Q6581072')]
[('prop', 'sex or gender'), ('gender', 'male'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('obj', 'http://www.wikidata.org/entity/Q6581097')]
[('prop', 'sex or gender'), ('gender', 'transgender female'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('obj', 'http://www.wikidata.org/entity/Q1052281')]
[('prop', 'sex or gender'), ('gender', 'transgender male'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('obj', 'http://www.wikidata.org/entity/Q2449503')]
[('prop', 'sex or gender'), ('gender', 'female organism'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('obj', 'http://www.wikidata.org/entity/Q43445')]
[('prop', 'sex or gender'), ('gender', 'intersex'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('obj', 'http://www.wikidata.org/entity/Q1097630')]
[('prop', 'sex or gender'), ('gender', 'non-binary'

12

#### Comment
Now we have the gender property and we save the "famale" node rearched through it:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P21`   | sex or gender   | predicate |
| `wd:Q6581072`     | female   | node |

In [34]:
# Let's try to merge the previous considerations
queryString = """
SELECT (COUNT(DISTINCT ?atf) AS ?frenchFemaleAthletes) (COUNT(DISTINCT ?atg) AS ?germanFemaleAthletes) WHERE {

{
?obj wdt:P31 wd:Q18536594 ;
    ?rel ?atf .    
?atf wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?nf ;
    wdt:P21 wd:Q6581072 .
    
FILTER(?nf = wd:Q142) .
FILTER(?rel = wdt:P1346) .

}UNION{

?obj wdt:P31 wd:Q18536594 ;
    ?rel ?atg .    
?atg wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?ng ;
    wdt:P21 wd:Q6581072 .
    
FILTER(?ng = wd:Q183) .
FILTER(?rel = wdt:P1346) .
}

}
"""

print("Results")
run_query(queryString)

Results
[('frenchFemaleAthletes', '31'), ('germanFemaleAthletes', '98')]


1

#### Comment
So it's higher the number of German female winners

### Task 4.3
4.3 For each country , which are the atheletes with the highest number of participations

#### Comment
Is the same as task 5.3 of the previous notebook (cdcf47393e), same problem in returning country and person.  
Even in this case I assume that for country they intend France and Germany.  
It's also not clear if the number of partecipation is referring to the editions of the Olympic Games or partecipation in referring to the competitions across different edition of the Olympic Games so I decide to assume this second hypotesis as the one that satisfy the task 4.3.   

In [65]:
#Find for each country (France and Germany) the highest number of partecipations for an athlete ofthe respective country
queryString = """
SELECT ?cname (MAX(?num) AS ?max)
WHERE { 
?at wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?cou ;
    ?p ?obj1 .
?obj1 wdt:P31 wd:Q18536594 .

FILTER(?at = ?a).
{
SELECT ?a (COUNT(DISTINCT ?obj) AS ?num)
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?country ;
    ?p1 ?obj .
?obj wdt:P31 wd:Q18536594 .

FILTER(?p1 = wdt:P710 || ?p1 = wdt:P1344) . # partecipant and partecipant in
    
FILTER(?country = wd:Q142 || ?country = wd:Q183) .
}
GROUP BY ?a
}

FILTER(?p = wdt:P710 || ?p = wdt:P1344) . # partecipant and partecipant in
FILTER(?cou = wd:Q142 || ?cou = wd:Q183) .

?at <http://schema.org/name> ?aname .
?cou <http://schema.org/name> ?cname .

}
GROUP BY ?cname
ORDER BY DESC (?max)
"""

print("Results")
run_query(queryString)

Results
[('cname', 'Germany'), ('max', '13')]
[('cname', 'France'), ('max', '11')]


2

#### Comment
In the COUNT() function I can use the DISTINCT directive because each competition is identified by the competition name, the speciality and the year of the Olympic Game in which it has been held.  
I can't print the name of the athlete because of the rules on aggregate functions...  
The alternative is to print the max for France in a query and the max for Germany in another but if the case is like the one in the notebook (cdcf47393e) I can't do this for many states one at a time... 

In [69]:
#Print the French athlete with the highest number of partecipations 
queryString = """
SELECT ?cname ?aname (COUNT(DISTINCT ?obj) AS ?num)
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?country ;
    ?p1 ?obj .
?obj wdt:P31 wd:Q18536594 .

FILTER(?p1 = wdt:P710 || ?p1 = wdt:P1344) . # partecipant and partecipant in
    
FILTER(?country = wd:Q142) .

?a <http://schema.org/name> ?aname .
?country <http://schema.org/name> ?cname .
}
GROUP BY ?aname ?cname
ORDER BY DESC (?num)
LIMIT 1

"""

print("Results")
run_query(queryString)

Results
[('cname', 'France'), ('aname', 'Charlotte Bonnet'), ('num', '11')]


1

In [68]:
#Print the German athlete with highest number of partecipations 
queryString = """
SELECT ?cname ?aname (COUNT(DISTINCT ?obj) AS ?num)
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?country ;
    ?p1 ?obj .
?obj wdt:P31 wd:Q18536594 .

FILTER(?p1 = wdt:P710 || ?p1 = wdt:P1344) . # partecipant and partecipant in
    
FILTER(?country = wd:Q183) .

?a <http://schema.org/name> ?aname .
?country <http://schema.org/name> ?cname .
}
GROUP BY ?aname ?cname
ORDER BY DESC (?num)
LIMIT 1

"""

print("Results")
run_query(queryString)

Results
[('cname', 'Germany'), ('aname', 'Birgit Fischer'), ('num', '13')]


1

#### Comment 
Let's try to verify the results using Charlotte Bonnet

In [3]:
#Print the French athlete with most partecipations printing his/her URI 
queryString = """
SELECT ?cname ?aname (COUNT(DISTINCT ?obj) AS ?num) ?a
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 ;
    wdt:P27 ?country ;
    ?p1 ?obj .
?obj wdt:P31 wd:Q18536594 .

FILTER(?p1 = wdt:P710 || ?p1 = wdt:P1344) . # partecipant and partecipant in
    
FILTER(?country = wd:Q142) .

?a <http://schema.org/name> ?aname .
?country <http://schema.org/name> ?cname .
}
GROUP BY ?aname ?cname ?a
ORDER BY DESC (?num)
LIMIT 1

"""

print("Results")
run_query(queryString)

Results
[('cname', 'France'), ('aname', 'Charlotte Bonnet'), ('num', '11'), ('a', 'http://www.wikidata.org/entity/Q129192')]


1

In [6]:
#Print the French athlete with most partecipations printing his/her URI 
queryString = """
SELECT ?aname ?pname ?oname
WHERE { 

wd:Q129192 ?p1 ?obj .
?obj wdt:P31 wd:Q18536594 .

FILTER(?p1 = wdt:P710 || ?p1 = wdt:P1344) . # partecipant and partecipant in

wd:Q129192 <http://schema.org/name> ?aname .
?p1 <http://schema.org/name> ?pname .
?obj <http://schema.org/name> ?oname .
}

"""

print("Results")
run_query(queryString)

Results
[('aname', 'Charlotte Bonnet'), ('pname', 'participant in'), ('oname', "swimming at the 2020 Summer Olympics – women's 4 × 100 metre freestyle relay")]
[('aname', 'Charlotte Bonnet'), ('pname', 'participant in'), ('oname', "swimming at the 2012 Summer Olympics – women's 100 metre freestyle")]
[('aname', 'Charlotte Bonnet'), ('pname', 'participant in'), ('oname', "swimming at the 2016 Summer Olympics – women's 200 metre freestyle")]
[('aname', 'Charlotte Bonnet'), ('pname', 'participant in'), ('oname', "swimming at the 2020 Summer Olympics – women's 200 metre freestyle")]
[('aname', 'Charlotte Bonnet'), ('pname', 'participant in'), ('oname', "swimming at the 2016 Summer Olympics – women's 4 × 200 metre freestyle relay")]
[('aname', 'Charlotte Bonnet'), ('pname', 'participant in'), ('oname', "swimming at the 2012 Summer Olympics – women's 4 × 200 metre freestyle relay")]
[('aname', 'Charlotte Bonnet'), ('pname', 'participant in'), ('oname', "swimming at the 2020 Summer Olympics –

11

#### Comment
Ok the results are correct and so we can say that previously we found the French and German athletes with more partecipations in Olympic Games competitions